In [ ]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

%matplotlib widget
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy
import cvxpy as cp

from interval import interval, inf, imath

from safe_train import *
from projection_examples import projection_training_loop

## Generate data

In [ ]:
x = np.linspace(-10, 10, 100)
y = np.linspace(-10, 10, 100)
x, y = np.meshgrid(x,y)
z = x**2 + y**2

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# plt.scatter(x, y, z)
ax.plot_surface(x, y, z)
# ax.set_zlim(-1, 101)
plt.show()

In [ ]:
x_array = x.reshape(-1, 1)
y_array = y.reshape(-1, 1)
z_array = z.reshape(-1, 1)
nn_x = np.hstack([x_array, y_array])

## Train network

In [ ]:
normalizer = layers.Normalization(
    input_shape=[
        2,
    ],
    axis=-1,
)
normalizer.adapt(nn_x)
regression_model = tf.keras.Sequential(
    [
        normalizer, 
        # layers.Dense(units=2, activation="linear", input_shape=[2]),
        layers.Dense(units=64, activation="relu"),
        layers.Dense(units=64, activation="relu"),
        layers.Dense(units=64, activation="relu"),
        layers.Dense(units=64, activation="relu"),
        layers.Dense(units=64, activation="relu"),
        # layers.Dense(units=10, activation="sigmoid"),
        layers.Dense(units=1, activation="linear"),
    ]
)
regression_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1), loss="mean_squared_error"
)

history = regression_model.fit(
    nn_x,
    z_array,
    epochs=10,
    # Suppress logging.
    verbose=1,
    # Calculate validation results on 10% of the training data.
    validation_split=0.1,
)

In [ ]:
plot_loss(history)

In [ ]:
z_predict = regression_model.predict(nn_x)
z_predict_mesh = z_predict.reshape(100, 100)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# plt.scatter(x, y, z)
surf1 = ax.plot_surface(x, y, z_predict_mesh)

surf1._edgecolors2d = surf1._edgecolor3d
surf1._facecolors2d = surf1._facecolor3d

plt.show()
# plt.plot(x, y_predict, color="C1")
# plt.scatter(x, y, color="C2")
# plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# plt.scatter(x, y, z)
surf1 = ax.scatter(x, y, z_predict_mesh)
surf2 = ax.scatter(x, y, z)

ax.legend(["NN predictions", "Data"])
plt.show()
# plt.plot(x, y_predict, color="C1")
# plt.scatter(x, y, color="C2")
# plt.show()

## Analyze safety with interval tool

In [ ]:
interval[-8, -5] ** 2 + interval[-8, -5] ** 2

In [ ]:
regression_model.predict([-8, -8])

In [ ]:
regression_model.predict([-5, -5])

In [ ]:
input_interval = [interval[-8, -5], interval[-8, -5]]
output_interval, penultimate_interval = propagate_interval(input_interval, regression_model)
output_interval

In [ ]:
x.reshape(-1).shape

In [ ]:
z_predict.shape

In [ ]:
plot_intervals(input_interval[0], output_interval, xlim = [-10, 10], ylim = [-10, 1000])

In [ ]:
plot_intervals(input_interval[0], output_interval, xs = x_array, ys = z_array, xlim = [-10, 10], ylim = [-10, 1000])

In [ ]:
plot_intervals(input_interval[0], output_interval, xs = x_array, ys = z_array, y_predict = z_predict, xlim = [-10, 10], ylim = [-10, 1000])

In [ ]:
plot_intervals(input_interval[0], output_interval, xs=x.reshape(-1), y_predict=z_predict, xlim=[-10, 10], ylim=[-5, 105])

FAILS: needs to handle multiple inputs well

In [ ]:
model = regression_model

In [ ]:
model.layers[0].get_weights()

In [ ]:
norm_mean, norm_var, _ = model.layers[0].get_weights()
norm_mean

In [ ]:
norm_var

## Simpler problem to troubleshoot

In [ ]:
x = np.linspace(-10, 10, 100)
y = np.linspace(-10, 10, 100)
z_func = lambda x, y: 2*x + y - 4

x, y = np.meshgrid(x,y)
z_array = z_func(x, y)

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# plt.scatter(x, y, z)
ax.scatter(x, y, z_array)
plt.show()

In [ ]:
x_array = x.reshape(-1, 1)
y_array = y.reshape(-1, 1)
z_array_long = z_array.reshape(-1, 1)
nn_x = np.hstack([x_array, y_array])

In [ ]:
normalizer = layers.Normalization(
    input_shape=[
        2,
    ],
    axis=-1,
)
normalizer.adapt(nn_x)
regression_model = tf.keras.Sequential(
    [
        normalizer, 
        layers.Dense(units=2, activation="linear", input_shape=[2]),
        # layers.Dense(units=64, activation="relu"),
        # layers.Dense(units=64, activation="relu"),
        # layers.Dense(units=64, activation="relu"),
        # layers.Dense(units=64, activation="relu"),
        # layers.Dense(units=64, activation="relu"),
        layers.Dense(units=1, activation="linear"),
    ]
)
regression_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1), loss="mean_squared_error"
)

history = regression_model.fit(
    nn_x,
    z_array_long,
    epochs=10,
    # Suppress logging.
    verbose=1,
    # Calculate validation results on 10% of the training data.
    validation_split=0.1,
)

In [ ]:
regression_model.predict([-8, -8])

In [ ]:
z_func(*[-8, -8])

In [ ]:
regression_model.predict([-5, -5])

In [ ]:
z_func(*[-5, -5])

In [ ]:
z_func(*[-1, -1])

In [ ]:
z_func(*[0, 0])

In [ ]:
input_interval = [interval[-8, -5], interval[-8, -5]]
output_interval, penultimate_interval = propagate_interval(input_interval, regression_model)
output_interval

In [ ]:
input_interval = [interval[0, inf], interval[-8, -5]]
output_interval, penultimate_interval = propagate_interval(input_interval, regression_model)
output_interval

In [ ]:
input_interval = [None, interval[-8, -5]]
output_interval, penultimate_interval = propagate_interval(input_interval, regression_model)
output_interval

## Try with one empty interval

## Test training
takeaway: make sure `shapes` match for `y, y_pred` passed to loss function. even `(100, 1)` and `(100,)` mismatch will lead to bad learning.

### Multiple regression: x, y, z

In [ ]:
x = np.linspace(-10, 10, 100)
y = np.linspace(-10, 10, 100)
x, y = np.meshgrid(x, y)
z = x**2 + y**2

x_array = x.reshape(-1, 1)
y_array = y.reshape(-1, 1)
z_array = z.reshape(-1, 1)
nn_x = np.hstack([x_array, y_array])

input_interval = [interval[-8, -5], interval[-8, -5]]
desired_interval = interval[-28, -19]

normalizer = layers.Normalization(
    input_shape=[
        2,
    ],
    axis=-1,
)
normalizer.adapt(nn_x)
regression_model = tf.keras.Sequential(
    [
        normalizer,
        # layers.Dense(units=2, activation="linear", input_shape=[2]),
        layers.Dense(units=64, activation="relu"),
        layers.Dense(units=64, activation="relu"),
        layers.Dense(units=64, activation="relu"),
        layers.Dense(units=64, activation="relu"),
        layers.Dense(units=64, activation="relu"),
        # layers.Dense(units=10, activation="sigmoid"),
        layers.Dense(units=1, activation="linear"),
    ]
)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)
loss_fn = tf.keras.losses.MeanSquaredError()

# don't project every epoch
EPOCH_TO_PROJECT = 5

epochs = 40
# epochs = 5
for epoch in range(epochs):
    print(f"\nStart of epoch {epoch}")

    with tf.GradientTape() as tape:
        z_pred = regression_model(nn_x, training=True)  # Forward pass
        # Compute the loss value
        # (the loss function is configured in `compile()`)
        loss = loss_fn(z_array, z_pred)

    # Compute gradients
    trainable_vars = regression_model.trainable_variables
    gradients = tape.gradient(loss, trainable_vars)
    # print(gradients)
    # print(trainable_vars)
    # Update weights
    optimizer.apply_gradients(zip(gradients, trainable_vars))

    output_interval, penultimate_interval = propagate_interval(
        input_interval, regression_model, graph=False
    )
    if type(output_interval) is list:
        if len(output_interval) == 1:
            output_interval = output_interval[0]
        else:
            raise NotImplementedError("Output interval was interval of length > 1")
    if output_interval not in desired_interval:
        print(f"safe region test FAILED, interval was {output_interval}")
        print(regression_model.layers[-1].weights)
        if epoch % EPOCH_TO_PROJECT == 0:
            print(f"\nProjecting weights at epoch {epoch}.")
            weights = regression_model.layers[-1].weights
            print(
                f"Old weights: {np.squeeze(np.array([weight.numpy() for weight in weights]))}"
            )
            projected_weights = project_weights(
                desired_interval,
                penultimate_interval,
                np.squeeze(np.array(weights)),
            )
            print(
                f"Projected weights: {projected_weights} yield new interval: "
                f"{penultimate_interval * projected_weights[0] + projected_weights[1]}"
            )
            proj_weight, proj_bias = projected_weights
            regression_model.layers[-1].set_weights(
                [np.array([[proj_weight]]), np.array([proj_bias])]
            )
            # NOTE: assume positive weights
            # TODO: handle both signs of weights

            # print(optimizer.get_weights())
            # optimizer.set_weights(last_safe_weights)
    else:
        print(f"safe region test passed, interval was {output_interval}")

    # Update metrics (includes the metric that tracks the loss)
    regression_model.compiled_metrics.update_state(y, y_pred)
    # Return a dict mapping metric names to current value

In [ ]:
test_interval = [interval([0.17749245743887973, 40.715509778405405]), interval([1.3693407866030003, 39.00241526956424]), interval([0.0]), interval([0.0, 2.8528582891451904]), interval([0.0, 0.8297828604382264]), interval([0.0, 26.46248532465599]), interval([0.0]), interval([5.1643618056622325, 51.965985161668776]), interval([0.0, 4.449784483001119]), interval([0.0, 12.181961657602645]), interval([1.1487307343228501, 36.84628332367279]), interval([0.0]), interval([0.0, 10.67893006000102]), interval([0.0, 32.31041089071087]), interval([0.0, 4.428521101926773]), interval([0.0, 33.27413406123338]), interval([0.0, 1.173106527682179]), interval([0.0, 5.763051345125981]), interval([0.0]), interval([0.0, 0.0593642406942681]), interval([1.1042434945592472, 42.40602482858007]), interval([0.0, 48.153185864747876]), interval([0.0, 2.093169161648054]), interval([0.0, 1.5303758911667666]), interval([1.5546250524211334, 41.40633134363574]), interval([1.2690299589762726, 44.50031948515337]), interval([0.0, 8.2281692197887]), interval([0.0, 0.3814058534273029]), interval([0.0, 29.90513713529191]), interval([0.0, 33.562458829446015]), interval([0.0, 0.7613352551489034]), interval([0.0, 38.30264206972309]), interval([0.0]), interval([0.0, 6.324824304874891]), interval([2.4997606995947597, 37.87439927718122]), interval([2.5371649489848416, 42.506682277456136]), interval([0.002846090267297985, 42.09635439199396]), interval([0.0]), interval([0.0, 0.6057659663937741]), interval([0.0, 12.80103993487166]), interval([0.0, 19.2486105128124]), interval([0.0, 37.93811481829238]), interval([2.968866596288945, 45.56460879954275]), interval([4.537518103822291, 51.34208899504068]), interval([0.0, 10.83174511415565]), interval([0.0, 0.2774448181623927]), interval([0.0]), interval([0.0, 29.00540786959906]), interval([0.08301538662180619, 35.51933660793823]), interval([1.0847326840759233, 38.7511647446967]), interval([0.0, 39.20136368640528]), interval([0.0, 42.341402156523365]), interval([0.0, 34.25962623911843]), interval([0.0, 48.34615648698638]), interval([2.4355944472457822, 39.57223969649944]), interval([0.0, 13.039277868905055]), interval([0.0, 4.195586380412922]), interval([0.0]), interval([0.0, 3.050183749047207]), interval([0.0]), interval([0.0, 0.1519885564424282]), interval([0.6696018918771051, 34.230603299831806]), interval([0.0, 30.686249110719995]), interval([5.5374481194296585, 50.683586670818116])]
len(test_interval)

### 2 dimensional

In [ ]:
x, y = generate_data(NOISE_STD=0.5, n=128)
plt.figure()
plt.scatter(x, y)
plt.show()

### With `model.fit`

Works perfectly, as expected

In [ ]:
normalizer = layers.Normalization(
    input_shape=[
        1,
    ],
    axis=None,
)
normalizer.adapt(x)
regression_model = tf.keras.Sequential(
    [
        normalizer, 
        layers.Dense(units=1, activation="linear", input_shape=[1]),
        layers.Dense(units=64, activation="relu"),
        layers.Dense(units=64, activation="relu"),
        layers.Dense(units=64, activation="relu"),
        # layers.Dense(units=10, activation="sigmoid"),
        layers.Dense(units=1, activation="linear"),
    ]
)
regression_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1), loss="mean_squared_error"
)
history = regression_model.fit(
    x,
    y,
    epochs=100,
    # Suppress logging.
    verbose=0,
    # Calculate validation results on 10% of the training data.
    validation_split=0.1,
)

plot_loss(history)

y_pred = regression_model.predict(x)

plt.figure()
plt.scatter(x, y_pred)
plt.scatter(x, y)
plt.legend(["nn", "data"])
plt.show()

### Manual

#### Batching

In [ ]:
## set up batches
rng = np.random.default_rng()
batch_size = 32

train_dataset = np.vstack([x,y]).T
train_dataset

rng.shuffle(train_dataset)  # in-place

x_shuffled = train_dataset[:, 0]
y_shuffled = train_dataset[:, 1]

x_batched = np.split(x_shuffled, np.arange(batch_size,len(x_shuffled),batch_size))
y_batched = np.split(y_shuffled, np.arange(batch_size,len(y_shuffled),batch_size))

dataset_batched = list(zip(x_batched, y_batched))

#### Train loop

In [ ]:
normalizer = layers.Normalization(
    input_shape=[
        1,
    ],
    axis=None,
)
normalizer.adapt(x)
regression_model = tf.keras.Sequential(
    [
        normalizer, 
        layers.Dense(units=1, activation="linear", input_shape=[1]),
        layers.Dense(units=64, activation="relu"),
        layers.Dense(units=64, activation="relu"),
        layers.Dense(units=64, activation="relu"),
        layers.Dense(units=4, activation="relu"),
        # layers.Dense(units=10, activation="sigmoid"),
        layers.Dense(units=1, activation="linear"),
    ]
)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)
loss_fn = tf.keras.losses.MeanSquaredError()

# don't project every epoch
EPOCH_TO_PROJECT = 5

# epochs = 500
epochs = 50
epoch_fraction = 0.1
report_every_epoch = int(epochs * epoch_fraction)
for epoch in range(epochs):
    # if epoch % report_every_epoch == 0:
        # print(f"on epoch {epoch}")
    for step, (x_batch_train, y_batch_train) in enumerate(dataset_batched):
        with tf.GradientTape() as tape:
            y_pred = regression_model(x_batch_train, training=True)  # Forward pass
            # IMPORTANT: squeeze to match dimensions before passing to loss
            y_pred = tf.squeeze(y_pred)
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = loss_fn(y_batch_train, y_pred)

        # Compute gradients
        trainable_vars = regression_model.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # print(gradients)
        # print(trainable_vars)
        # Update weights
        optimizer.apply_gradients(zip(gradients, trainable_vars))
# print(f"done training")
        
y_pred = regression_model.predict(x)

plt.figure()
plt.scatter(x, y_pred)
plt.scatter(x, y)
plt.legend(["nn", "data"])
plt.show()        

In [ ]:
out, penum = propagate_interval([interval[10, 20]], regression_model)
out

In [ ]:
penum

In [ ]:
# true function
interval[10, 20] * 0.5 + 5

## Projection with multiple final nodes

In [ ]:
input_intervals = penum
input_intervals

In [ ]:
weights = regression_model.layers[-1].weights
weights

In [ ]:
theta = np.squeeze(np.vstack([weight.numpy() for weight in weights]).flatten())
theta

In [ ]:
goal_interval = interval[10, 15]
goal_interval

In [ ]:
proj = project_weights(goal_interval, input_intervals, theta, verbose=False)
proj

In [ ]:
proj_out = 0
for i, weight in enumerate(proj[:-1]):
    proj_out += input_intervals[i] * weight
proj_out += proj[-1]
proj_out

In [ ]:
input_intervals = [interval[28.3, 49.1], interval[1, 3]]

In [ ]:
# input_intervals = [interval[28.3, 49.1], interval[2, 5]]

In [ ]:
theta = np.array([0.148, 0.377, 1.79])

In [ ]:
input_intervals[0] * theta[0] + input_intervals[1]*theta[1] + theta[2]

In [ ]:
x = cp.Variable(theta.shape)
x

In [ ]:
import itertools

test = list(itertools.product(*[(elem[0]) for elem in input_intervals]))
test

In [ ]:
[np.hstack([elem, 1]) for elem in test]

In [ ]:
goal_interval[0][0]

In [ ]:
goal_interval[0][1]

In [ ]:
def generate_constraints(input_intervals, goal_interval, x):
    interval_combinations = list(itertools.product(
        *[(elem[0]) for elem in input_intervals]))
    constraint_vectors = [np.hstack([elem, 1]) for elem in interval_combinations]
    constraints = []
    print("Generating constraints:")
    for i, constraint_vector in enumerate(constraint_vectors):
        print(f"{constraint_vector} @ x >= {goal_interval[0][0]}")
        constraints.append(constraint_vector @ x >= goal_interval[0][0])
        print(f"{constraint_vector} @ x <= {goal_interval[0][1]}")
        constraints.append(constraint_vector @ x <= goal_interval[0][1])
    return constraints

In [ ]:
input_intervals

In [ ]:
x = cp.Variable((3,))
constraints = generate_constraints(input_intervals, goal_interval, x)
# constraints = [
#     np.array([28.3, 1, 1]) @ x >= 10,
#     np.array([49.1, 3, 1]) @ x <= 15,
#     np.array([28.3, 3, 1]) @ x >= 10,
#     np.array([49.1, 1, 1]) @ x <= 15,
# ]

obj = cp.Minimize(
    cp.norm(x - np.array([0.158, 0.377, 1.79]))
)
# Form and solve problem.
prob = cp.Problem(obj, constraints)
import time
tstart = time.time()
prob.solve()  # Returns the optimal value.
tfinish = time.time()
print("status:", prob.status)
print("optimal value", prob.value)
print(f"optimal variables are: {np.round(x.value, 5)}")
print(f"took {tfinish - tstart} seconds")

In [ ]:
shift_lower = np.array(([0] * len(input_intervals)) + [goal_interval[0].inf])
shift_lower

In [ ]:
direction_lower = np.array([1] + [-input_interval[0].inf for input_interval in input_intervals][::-1])
direction_lower

In [ ]:
project_lower = (
    (direction_lower @ (theta - shift_lower))
    / (direction_lower @ direction_lower)
    * direction_lower
)
project_lower

In [ ]:
param_lower = project_lower + shift_lower
param_lower

In [ ]:
shift_upper = np.array(([0] * len(input_intervals)) + [goal_interval[0].sup])
shift_upper

In [ ]:
# direction_upper = np.array([1] + [-input_interval[0].sup for input_interval in input_intervals])
direction_upper = np.array([1] + [-input_interval[0].sup for input_interval in input_intervals][::-1])
direction_upper

In [ ]:
project_upper = (
    (direction_upper @ (theta - shift_upper))
    / (direction_upper @ direction_upper)
    * direction_upper
)
project_upper

In [ ]:
param_upper = project_upper + shift_upper
param_upper

In [ ]:
theta

In [ ]:
best_params = min([param_upper, param_lower], key=lambda param: np.linalg.norm(theta - param))
best_params

In [ ]:
input_intervals[0] * best_params[0] + input_intervals[1] * best_params[1] + best_params[2]

In [ ]:
input_intervals = [interval[28.3, 49.1], interval[0]]
input_intervals

In [ ]:
best_params = [0.240385, 0.377, 3.19712]
best_params

In [ ]:
input_intervals[0] * 0.158 + 1.79

In [ ]:
input_intervals[0] * best_params[0] + input_intervals[1] * best_params[1] + best_params[2]

In [ ]:
interval[28.3, 49.1] * 0.289699 + interval[1,3] * -0.512874 + 2.31438

In [ ]:
interval[28.3, 49.1] * 0.240385 + interval[0] * .377 + 3.19712

In [ ]:
28.3 * 0.289699 + 1 * -0.512874 + 2.31438

In [ ]:
49.1 * 0.289699 + 3 * -0.512874 + 2.31438

In [ ]:
projection_training_loop()

## CVXpy

### individual variables

In [ ]:
x = cp.Variable()
y = cp.Variable()
z = cp.Variable()

constraints = [
    28.3 * x + 1 * y + z >= 10,
    49.1 * x + 3 * y + z <= 15,
    # 28.3 * x + 3 * y + z >= 10,
    # 49.1 * x + 1 * y + z <= 15
]

obj = cp.Minimize(
    (x - 0.158)**2 + (y - .377)**2 + (z - 1.79)**2
)
# Form and solve problem.
prob = cp.Problem(obj, constraints)
import time
tstart = time.time()
prob.solve()  # Returns the optimal value.
tfinish = time.time()
print("status:", prob.status)
print("optimal value", prob.value)
print(f"optimal variables are x: {np.round(x.value, 5)}, y: {np.round(y.value, 5)}, z: {np.round(z.value, 5)}")
print(f"took {tfinish - tstart} seconds")

### vectorized version

In [ ]:
x = cp.Variable((3,))

constraints = [
    np.array([28.3, 1, 1]) @ x >= 10,
    np.array([49.1, 3, 1]) @ x <= 15,
    # 28.3 * x + 3 * y + z >= 10,
    # 49.1 * x + 1 * y + z <= 15
]

obj = cp.Minimize(
    cp.norm(x - np.array([0.158, 0.377, 1.79]))
)
# Form and solve problem.
prob = cp.Problem(obj, constraints)
import time
tstart = time.time()
prob.solve()  # Returns the optimal value.
tfinish = time.time()
print("status:", prob.status)
print("optimal value", prob.value)
print(f"optimal values are: {np.round(x.value, 5)}")
print(f"took {tfinish - tstart} seconds")

In [ ]:
x = cp.Variable((3,))

constraints = [
    np.array([28.3, 1, 1]) @ x >= 10,
    np.array([49.1, 3, 1]) @ x <= 15,
    np.array([28.3, 3, 1]) @ x >= 10,
    np.array([49.1, 1, 1]) @ x <= 15,
]

obj = cp.Minimize(
    cp.norm(x - np.array([0.158, 0.377, 1.79]))
)
# Form and solve problem.
prob = cp.Problem(obj, constraints)
import time
tstart = time.time()
prob.solve()  # Returns the optimal value.
tfinish = time.time()
print("status:", prob.status)
print("optimal value", prob.value)
print(f"optimal variables are x: {np.round(x.value, 5)}, y: {np.round(y.value, 5)}, z: {np.round(z.value, 5)}")
print(f"took {tfinish - tstart} seconds")

In [ ]:
interval[28.3, 49.1] * x.value + interval[1,3] * y.value + z.value

In [ ]:
x = cp.Variable()
y = cp.Variable()
z = cp.Variable()

constraints = [
    28.3 * x + 1 * y + z >= 10,
    49.1 * x + 3 * y + z <= 15,
    28.3 * x + 3 * y + z >= 10,
    49.1 * x + 1 * y + z <= 15
]

obj = cp.Minimize(
    (x - 0.158)**2 + (y - .377)**2 + (z - 1.79)**2
)
# Form and solve problem.
prob = cp.Problem(obj, constraints)
import time
tstart = time.time()
prob.solve()  # Returns the optimal value.
tfinish = time.time()
print("status:", prob.status)
print("optimal value", prob.value)
print(f"optimal variables are x: {np.round(x.value, 5)},",
       "y: {np.round(y.value, 5)},",
       "z: {np.round(z.value, 5)}")
print(f"took {tfinish - tstart} seconds")

In [ ]:
interval[28.3, 49.1] * x.value + interval[1,3] * y.value + z.value